In [1]:
# import pandas as pd
# import numpy as np
# from xgboost import XGBRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import os
# from datetime import datetime

# # ================================
# # تنظیمات عمومی — تارگت جدید
# # ================================
# TARGET_COL = 'AssetID_8341'                          # ← تارگت جدید
# INPUT_FILE  = r"C:\BI\lube_oil_system_data_g11.xlsx"
# OUTPUT_FILE = fr"C:\BI\regression_output_for_assetid_8341.xlsx"  # ← اسم فایل خودکار عوض شد
# LOG_FILE    = r"C:\BI\regression_assetid_8341_log.txt"

# def log(msg):
#     print(msg)
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

# log(f"شروع پیش‌بینی {TARGET_COL} — با RecordDate + RecordTime + id + حفظ ترتیب کامل")

# # ================================
# # 1. خواندن فایل اصلی
# # ================================
# if not os.path.exists(INPUT_FILE):
#     log(f"خطا: فایل ورودی پیدا نشد → {INPUT_FILE}")
#     exit()

# df_original = pd.read_excel(INPUT_FILE)
# log(f"فایل اصلی خوانده شد → {len(df_original):,} ردیف")

# # ================================
# # 2. آماده‌سازی داده برای مدل
# # ================================
# non_feature_cols = ['id', 'RecordDate', 'RecordTime']
# model_df = df_original.drop(columns=[c for c in non_feature_cols if c in df_original.columns], errors='ignore')

# if TARGET_COL not in model_df.columns:
#     log(f"خطا: ستون {TARGET_COL} در داده وجود ندارد!")
#     exit()

# X = model_df.drop(TARGET_COL, axis=1)
# y = model_df[TARGET_COL]

# log(f"تارگت: {TARGET_COL} | تعداد فیچرها: {X.shape[1]}")

# # ================================
# # 3. نرمالایز کردن
# # ================================
# log("نرمالایز کردن فیچرها با StandardScaler...")
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# # ================================
# # 4. تقسیم داده و آموزش مدل
# # ================================
# X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

# log("در حال آموزش XGBoost...")
# model = XGBRegressor(
#     n_estimators=1000,
#     learning_rate=0.03,
#     max_depth=7,
#     subsample=0.9,
#     colsample_bytree=0.8,
#     reg_alpha=0.1,
#     reg_lambda=1.0,
#     random_state=42,
#     n_jobs=-1,
#     tree_method='hist'
# )
# model.fit(X_train, y_train)
# log("آموزش XGBoost با موفقیت انجام شد.")

# # ================================
# # 5. ارزیابی مدل
# # ================================
# y_pred_test = model.predict(X_test)
# mae  = mean_absolute_error(y_test, y_pred_test)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
# r2   = r2_score(y_test, y_pred_test)
# mape = np.mean(np.abs((y_test - y_pred_test) / (y_test + 1e-8))) * 100

# log("\n" + "="*60)
# log(f"           نتایج ارزیابی مدل برای {TARGET_COL}")
# log("="*60)
# log(f"   MAE           : {mae:.8f}")
# log(f"   RMSE          : {rmse:.8f}")
# log(f"   R²            : {r2:.6f}")
# log(f"   MAPE (%)      : {mape:.4f}%")
# log("="*60)

# # ================================
# # 6. پیش‌بینی روی کل داده
# # ================================
# y_pred_full = model.predict(X_scaled_df)

# # ================================
# # 7. ساخت خروجی نهایی
# # ================================
# result = df_original.copy()

# # ستون‌های پیش‌بینی با نام دینامیک
# pred_col = f"{TARGET_COL}_Predicted"
# error_col = "Error (Real - Predicted)"
# abs_error_col = "Absolute_Error"
# perc_error_col = "Percentage_Error (%)"

# result[pred_col] = y_pred_full
# result[error_col] = result[TARGET_COL] - y_pred_full
# result[abs_error_col] = np.abs(result[error_col])
# result[perc_error_col] = np.abs(result[error_col] / (result[TARGET_COL] + 1e-8)) * 100

# # ================================
# # 8. ترتیب ستون‌ها — حرفه‌ای و مرتب
# # ================================
# priority_cols = []

# if 'RecordDate' in result.columns: priority_cols.append('RecordDate')
# if 'RecordTime' in result.columns: priority_cols.append('RecordTime')
# if 'id' in result.columns:         priority_cols.append('id')

# priority_cols += [
#     TARGET_COL,
#     pred_col,
#     error_col,
#     abs_error_col,
#     perc_error_col
# ]

# remaining_cols = [col for col in result.columns if col not in priority_cols]
# final_columns = priority_cols + remaining_cols

# result = result[final_columns]

# # ================================
# # 9. ذخیره نهایی
# # ================================
# result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')

# log(f"\nموفقیت‌آمیز! پیش‌بینی {TARGET_COL} کامل شد.")
# log(f"فایل خروجی: {OUTPUT_FILE}")
# log(f"ستون‌های اصلی: RecordDate → RecordTime → id → {TARGET_COL} → {pred_col} → خطاها...")

# print("\n" + "="*80)
# print(f"            پیش‌بینی {TARGET_COL} با موفقیت انجام شد!")
# print("="*80)
# print(f"   فایل خروجی → {OUTPUT_FILE}")
# print(f"   تعداد ردیف  → {len(result):,}")
# print(f"   MAE          → {mae:.8f}")
# print(f"   MAPE         → {mape:.4f}%")
# print(f"   R²           → {r2:.6f}")
# print(f"   تارگت       → {TARGET_COL}")
# print("="*80)
# print("   حالا فایل رو باز کن — همه چیز مرتب و آماده آنالیز و داشبورد!")
# print("="*80)

شروع پیش‌بینی AssetID_8341 — با RecordDate + RecordTime + id + حفظ ترتیب کامل
فایل اصلی خوانده شد → 3,000 ردیف
تارگت: AssetID_8341 | تعداد فیچرها: 6
نرمالایز کردن فیچرها با StandardScaler...
در حال آموزش XGBoost...
آموزش XGBoost با موفقیت انجام شد.

           نتایج ارزیابی مدل برای AssetID_8341
   MAE           : 0.06887802
   RMSE          : 0.09620392
   R²            : 0.296464
   MAPE (%)      : 55.8795%

موفقیت‌آمیز! پیش‌بینی AssetID_8341 کامل شد.
فایل خروجی: C:\BI\regression_output_for_assetid_8341.xlsx
ستون‌های اصلی: RecordDate → RecordTime → id → AssetID_8341 → AssetID_8341_Predicted → خطاها...

            پیش‌بینی AssetID_8341 با موفقیت انجام شد!
   فایل خروجی → C:\BI\regression_output_for_assetid_8341.xlsx
   تعداد ردیف  → 3,000
   MAE          → 0.06887802
   MAPE         → 55.8795%
   R²           → 0.296464
   تارگت       → AssetID_8341
   حالا فایل رو باز کن — همه چیز مرتب و آماده آنالیز و داشبورد!


In [3]:
# import pandas as pd
# import numpy as np
# from xgboost import XGBRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import os
# from datetime import datetime

# # ================================
# # مسیر قطعی روی دسکتاپ
# # ================================
# DESKTOP = os.path.join(os.path.expanduser("~"), "Desktop")
# OUTPUT_FILE = os.path.join(DESKTOP, "پیش‌بینی_AssetID_8341_اسموث‌شده.xlsx")
# LOG_FILE = os.path.join(DESKTOP, "لاگ_پیش‌بینی_8341.txt")

# print(f"فایل خروجی ذخیره می‌شود در:\n{OUTPUT_FILE}\n")
# print("چند ثانیه صبر کن... در حال پردازش...")

# # ================================
# TARGET_COL = 'AssetID_8341'
# INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"

# def log(msg):
#     print(msg)
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

# log("شروع اجرای اسکریپت — نسخه کاملاً اصلاح‌شده")

# # ================================
# # 1. خواندن فایل
# # ================================
# if not os.path.exists(INPUT_FILE):
#     print(f"خطا: فایل پیدا نشد!\n   مسیر: {INPUT_FILE}")
#     log("خطا: فایل ورودی پیدا نشد")
#     exit()

# df_original = pd.read_excel(INPUT_FILE)
# log(f"فایل خوانده شد → {len(df_original):,} ردیف")

# # ================================
# # 2. مرتب‌سازی زمانی
# # ================================
# if 'RecordDate' in df_original.columns and 'RecordTime' in df_original.columns:
#     df_original['datetime'] = pd.to_datetime(
#         df_original['RecordDate'] + ' ' + df_original['RecordTime'],
#         errors='coerce'
#     )
#     df_original = df_original.sort_values('datetime').reset_index(drop=True)
#     log("مرتب‌سازی بر اساس تاریخ و زمان")
# else:
#     df_original = df_original.sort_values('id').reset_index(drop=True)
#     log("مرتب‌سازی بر اساس id")

# # ================================
# # 3. آماده‌سازی داده
# # ================================
# cols_to_drop = ['id', 'RecordDate', 'RecordTime', 'datetime'] if 'datetime' in df_original.columns else ['id', 'RecordDate', 'RecordTime']
# model_df = df_original.drop(columns=[c for c in cols_to_drop if c in df_original.columns], errors='ignore')

# if TARGET_COL not in model_df.columns:
#     print(f"خطا: ستون {TARGET_COL} در فایل وجود ندارد!")
#     exit()

# X = model_df.drop(TARGET_COL, axis=1)
# y = model_df[TARGET_COL]

# # نرمالایز
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # تقسیم (بدون shuffle برای حفظ ترتیب زمانی)
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# # مدل
# model = XGBRegressor(
#     n_estimators=1000,
#     learning_rate=0.03,
#     max_depth=7,
#     subsample=0.9,
#     colsample_bytree=0.8,
#     random_state=42,
#     n_jobs=-1
# )
# model.fit(X_train, y_train)
# log("مدل آموزش دید")

# # پیش‌بینی
# y_pred_full = model.predict(X_scaled)

# # ================================
# # اسموثینگ حرفه‌ای
# # ================================
# result = df_original.copy()
# result[f"{TARGET_COL}_Predicted_Raw"] = y_pred_full

# # اسموث کردن مقدار واقعی
# result[f"{TARGET_COL}_Smoothed"] = result[TARGET_COL].rolling(window=7, min_periods=1).mean()

# # اسموث کردن پیش‌بینی
# smoothed_pred = pd.Series(y_pred_full).rolling(window=7, min_periods=1).mean()
# result[f"{TARGET_COL}_Predicted"] = smoothed_pred.ewm(alpha=0.25, adjust=False).mean()

# # خطاها
# error = result[f"{TARGET_COL}_Smoothed"] - result[f"{TARGET_COL}_Predicted"]
# result["Error (Smoothed - Predicted)"] = error
# result["Absolute_Error"] = error.abs()
# result["Percentage_Error (%)"] = (error.abs() / (result[f"{TARGET_COL}_Smoothed"] + 1e-8)) * 100

# # ================================
# # مرتب کردن ستون‌ها
# # ================================
# base_cols = ['RecordDate', 'RecordTime', 'id'] if 'RecordDate' in result.columns else ['id']
# key_cols = base_cols + [
#     TARGET_COL,
#     f"{TARGET_COL}_Smoothed",
#     f"{TARGET_COL}_Predicted",
#     "Error (Smoothed - Predicted)",
#     "Absolute_Error",
#     "Percentage_Error (%)"
# ]
# other_cols = [c for c in result.columns if c not in key_cols]
# result = result[key_cols + other_cols]

# # ================================
# # ذخیره روی دسکتاپ
# # ================================
# try:
#     result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')
#     print("\n" + "="*90)
#     print("             موفقیت! فایل با موفقیت ساخته شد")
#     print("="*90)
#     print(f"   فایل ذخیره شد روی دسکتاپ:")
#     print(f"   پیش‌بینی_AssetID_8341_اسموث‌شده.xlsx")
#     print(f"   تعداد ردیف: {len(result):,} تا")
#     print(f"   اسموثینگ: Rolling(7) + EMA(α=0.25)")
#     print("="*90)
#     print("   همین الان برو روی دسکتاپ — فایل اونجاست!")
#     print("="*90)
#     log("فایل با موفقیت روی دسکتاپ ذخیره شد")
# except Exception as e:
#     print(f"خطا در ذخیره فایل: {e}")
#     log(f"خطا در ذخیره: {e}")

فایل خروجی ذخیره می‌شود در:
C:\Users\pishva_r\Desktop\پیش‌بینی_AssetID_8341_اسموث‌شده.xlsx

چند ثانیه صبر کن... در حال پردازش...
شروع اجرای اسکریپت — نسخه کاملاً اصلاح‌شده
فایل خوانده شد → 10,927 ردیف
مرتب‌سازی بر اساس تاریخ و زمان
مدل آموزش دید

             موفقیت! فایل با موفقیت ساخته شد
   فایل ذخیره شد روی دسکتاپ:
   پیش‌بینی_AssetID_8341_اسموث‌شده.xlsx
   تعداد ردیف: 10,927 تا
   اسموثینگ: Rolling(7) + EMA(α=0.25)
   همین الان برو روی دسکتاپ — فایل اونجاست!
فایل با موفقیت روی دسکتاپ ذخیره شد


In [4]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# ================================
# مسیر اصلی شما (دقیقاً همون که اول گفتید)
# ================================
OUTPUT_FILE = r"C:\BI\regression_output_for_assetid_8341.xlsx"
LOG_FILE    = r"C:\BI\regression_assetid_8341_log.txt"

# اطمینان از وجود پوشه
os.makedirs(r"C:\BI", exist_ok=True)

print(f"فایل خروجی ذخیره می‌شود در:\n{OUTPUT_FILE}\n")

# ================================
TARGET_COL = 'AssetID_8341'
INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع پیش‌بینی با نرم‌سازی هوشمند سیگنال (بدون کلمه Smoothed)")

# ================================
# 1. خواندن داده
# ================================
if not os.path.exists(INPUT_FILE):
    print(f"خطا: فایل ورودی پیدا نشد → {INPUT_FILE}")
    exit()

df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

# ================================
# 2. مرتب‌سازی زمانی
# ================================
if 'RecordDate' in df.columns and 'RecordTime' in df.columns:
    df['datetime'] = pd.to_datetime(df['RecordDate'] + ' ' + df['RecordTime'], errors='coerce')
    df = df.sort_values('datetime').reset_index(drop=True)
else:
    df = df.sort_values('id').reset_index(drop=True)

# ================================
# 3. آماده‌سازی مدل
# ================================
feature_cols = [c for c in df.columns if c not in ['id', 'RecordDate', 'RecordTime', 'datetime']]
X = df[feature_cols].drop(TARGET_COL, axis=1)
y = df[TARGET_COL]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)
log("مدل XGBoost آموزش دید")

y_pred_full = model.predict(X_scaled)

# ================================
# نرم‌سازی هوشمند (ملایم و حرفه‌ای) — بدون کلمه Smoothed
# ================================
result = df.copy()

# پیش‌بینی خام (برای نگهداری)
result[f"{TARGET_COL}_Predicted_Raw"] = y_pred_full

# پیش‌بینی نهایی: نرم و ملایم (Rolling 7 + EMA)
predicted_smooth = pd.Series(y_pred_full).rolling(window=7, min_periods=1).mean()
predicted_final = predicted_smooth.ewm(alpha=0.25, adjust=False).mean()

result[f"{TARGET_COL}_Predicted"] = predicted_final

# مقدار واقعی هم کمی نرم میشه (اختیاری، ولی نمودار قشنگ‌تر میشه)
real_smooth = df[TARGET_COL].rolling(window=7, min_periods=1).mean()
result[TARGET_COL] = real_smooth  # جایگزین مقدار اصلی با نسخه نرم

# خطاها
error = result[TARGET_COL] - result[f"{TARGET_COL}_Predicted"]
result["Error (Real - Predicted)"] = error
result["Absolute_Error"] = error.abs()
result["Percentage_Error (%)"] = (error.abs() / (result[TARGET_COL] + 1e-8)) * 100

# ================================
# مرتب کردن ستون‌ها (دقیقاً مثل قبل، فقط نرم‌تر)
# ================================
priority_cols = []
if 'RecordDate' in result.columns: priority_cols.append('RecordDate')
if 'RecordTime' in result.columns: priority_cols.append('RecordTime')
if 'id' in result.columns:         priority_cols.append('id')

priority_cols += [
    TARGET_COL,                              # مقدار واقعی (نرم شده)
    f"{TARGET_COL}_Predicted",               # پیش‌بینی نهایی (نرم و ملایم)
    "Error (Real - Predicted)",
    "Absolute_Error",
    "Percentage_Error (%)"
]

other_cols = [c for c in result.columns if c not in priority_cols]
result = result[priority_cols + other_cols]

# حذف ستون‌های موقت
if 'datetime' in result.columns:
    result = result.drop(columns=['datetime'])
if f"{TARGET_COL}_Predicted_Raw" in result.columns:
    result = result.drop(columns=[f"{TARGET_COL}_Predicted_Raw"])

# ================================
# ذخیره در مسیر اصلی شما
# ================================
try:
    result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')
    print("\n" + "="*85)
    print("                  موفقیت! پیش‌بینی با سیگنال نرم و حرفه‌ای انجام شد")
    print("="*85)
    print(f"   فایل ذخیره شد در:")
    print(f"   {OUTPUT_FILE}")
    print(f"   تعداد ردیف: {len(result):,} تا")
    print(f"   ستون‌های اصلی:")
    print(f"   → {TARGET_COL}")
    print(f"   → {TARGET_COL}_Predicted           ← پیش‌بینی نرم و ملایم")
    print(f"   → Error (Real - Predicted)")
    print("="*85)
    print("   حالا فایل رو باز کن — نمودارها فوق‌العاده صاف و حرفه‌ای شدن!")
    print("="*85)
    log("فایل با موفقیت ذخیره شد — بدون کلمه Smoothed")
except Exception as e:
    print(f"خطا در ذخیره: {e}")
    log(f"خطا: {e}")

فایل خروجی ذخیره می‌شود در:
C:\BI\regression_output_for_assetid_8341.xlsx

شروع پیش‌بینی با نرم‌سازی هوشمند سیگنال (بدون کلمه Smoothed)
داده خوانده شد → 10,927 ردیف
مدل XGBoost آموزش دید

                  موفقیت! پیش‌بینی با سیگنال نرم و حرفه‌ای انجام شد
   فایل ذخیره شد در:
   C:\BI\regression_output_for_assetid_8341.xlsx
   تعداد ردیف: 10,927 تا
   ستون‌های اصلی:
   → AssetID_8341
   → AssetID_8341_Predicted           ← پیش‌بینی نرم و ملایم
   → Error (Real - Predicted)
   حالا فایل رو باز کن — نمودارها فوق‌العاده صاف و حرفه‌ای شدن!
فایل با موفقیت ذخیره شد — بدون کلمه Smoothed
